In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [2]:
def get_bits(x, n):
    return [int(x) for x in '{:0{size}b}'.format(x, size=n)]

In [3]:
def generate_sumator(n):
    qrX = QuantumRegister(n, name='x') # input register
    qrY = QuantumRegister(n, name='y') # output register, contains result of (x+y) mod n
    qrC = QuantumRegister(1, name='c') # output register, contains carry of (x+y)
    
    qc = QuantumCircuit(qrY, qrC, qrX)

    # 1
    for i in range(1, n): # 1..n-1
        qc.cx(qrX[i], qrY[i])

    # 2
    qc.cx(qrX[n-1], qrC)
    for i in range(n-2, 0, -1): # n-2..1
        qc.cx(qrX[i], qrX[i+1])

    # 3    
    for i in range(0, n-1):
        qc.ccx(qrX[i], qrY[i], qrX[i+1])
    qc.ccx(qrX[n-1], qrY[n-1], qrC)

    # 4
    for i in range(n-1, 0, -1):
        qc.cx(qrX[i], qrY[i])
        qc.ccx(qrX[i-1], qrY[i-1], qrX[i])

    # 5
    for i in range(1, n-1):
        qc.cx(qrX[i], qrX[i+1])

    # 6
    for i in range(0, n):
        qc.cx(qrX[i], qrY[i])
        
    return qc

In [4]:
# qc = generate_sumator(5)
# qc.draw(output='mpl', fold=-1)

In [5]:
def generate_init_part(x, y, n):
    x_bits = get_bits(x, n)
    y_bits = get_bits(y, n)
    
    qrX = QuantumRegister(n, name='x')
    qrY = QuantumRegister(n, name='y')
    qc = QuantumCircuit(qrY, qrX)
    
    for i in range(n):
        if x_bits[n-(i+1)] == 1:
            qc.x(qrX[i])

    for i in range(n):
        if y_bits[n-(i+1)] == 1:
            qc.x(qrY[i])

    qc.barrier(qrX[:], qrY[:])
    
    return qc

In [6]:
# qc_init = generate_init_part(1, 3, 2)
# qc_init.draw(output='mpl')

In [7]:
def generate_measure_part(n):
    qrX = QuantumRegister(n, name='x')
    qrY = QuantumRegister(n, name='y')
    qrC = QuantumRegister(1, name='c')
    crY = ClassicalRegister(n, name='yValue')
    crX = ClassicalRegister(n, name='xValue')
    crC = ClassicalRegister(1, name='cValue')

    qc = QuantumCircuit(qrY, qrC, qrX, crY, crC, crX)
    qc.barrier(qrY[:], qrC[:], qrX[:])
    qc.measure(qrY[:], crY[:])
    qc.measure(qrC[:], crC[:])
    qc.measure(qrX[:], crX[:])
    
    return qc

In [8]:
# qc_meas = generate_measure_part(2)
# qc_meas.draw(output='mpl')

In [9]:
def test_sum(x, y, n, actual_sum):
    expected_sum = (x + y) % (2 ** n)
    print(expected_sum, actual_sum, expected_sum == actual_sum)

In [10]:
# test_sum(6, 10, 4, 0)

In [11]:
def test_carry(x, y, n, carry):
    expected_carry = int((x + y) / (2 ** n))
    print(expected_carry, carry, expected_carry == carry)

In [12]:
from qiskit import Aer
from qiskit import execute

backend = Aer.get_backend('qasm_simulator')

In [13]:
def test(x, y, n):
    init = generate_init_part(x, y, n)
    sum_gate = generate_sumator(n)
    meas = generate_measure_part(n)

    qc = init + sum_gate + meas

    job = execute(qc, backend, shots=1)
    result = job.result()
    values = list(result.get_counts(qc).keys())

    assert len(values) == 1
    values = [int(v, 2) for v in values[0].split(' ')]

    c_value = values[1]
    x_value = values[0]
    y_value = values[2]

    print(x, x_value, x == x_value)
    test_sum(x, y, n, y_value)
    test_carry(x, y, n, c_value)

In [14]:
test(6, 10, 4)

<ipython-input-13-fd4ca9786444>:6: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc = init + sum_gate + meas
/home/bartek/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:869: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)


6 6 True
0 0 True
1 1 True


In [15]:
# for n in range(4):
#     n = n + 1
#     N = 2 ** n
#     for a in range(N):
#         for g in range(N):
#             print('---', a, g, '---')
#             test(a, g, n)